# L’analyse discriminante linéaire

Dans le chapitre précédent, nous avons abordé les techniques de regroupement d’observations dans des partitions (*clusters*) à des fins de classification. L’analyse discriminante linéaire (LDA pour *Linear Discriminant Analysis*) intervient en quelque sorte après, dans le sens où elle s’attache à repérer les structures qui ont autorisé l’attribution des classes (groupes).

Avant d’opérer une LDA, il faut s’assurer de respecter plusieurs contraintes :

- Le jeu de données est complet ;
- toutes les variables indépendantes sont continues (c’est-à-dire normalisées) ;
- les classes ont correctement été assignées.

Par ailleurs, une LDA repose sur deux hypothèses fondamentales :

- **La multinormalité :** les variables indépendantes sont normalement distribuées au sein des classes.
- **L’homoscédasticité :** les matrices de covariance des classes sont identiques.

Ajoutons à cela que la LDA cherche à maximiser la variance inter-classes en minimisant la variance intra-classe, aboutissant à une séparation linéaire entre les classes. De fait, si les données ne sont pas séparables linéairement, la LDA risque de ne pas bien fonctionner.

De nombreuses contraintes, qui ne doivent malgré tout pas empêcher d’utiliser cette technique : la LDA peut s’effectuer sur des données multi-classes et ne demande pas d’énormes calculs contrairement à d’autres comme la régression logistique pour des résultats équivalents.

## Présentation des données

Commençons par charger toutes les bibliothèques nécessaires pour ce projet :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

Prenons ensuite un jeu de données réduit issu d’une courte analyse de neuf fichiers appartenant au corpus *Brown*. L’objectif est de déterminer la meilleure manière d’analyser les données pour attribuer à coup sûr la bonne catégorie.

In [ ]:
data = {
    'Tokens': [2374, 2370, 2428, 2213, 2334, 2332, 2200, 2234, 2244],
    'Types': [745, 672, 737, 743, 735, 833, 956, 916, 908],
    'TTR': [0.313816, 0.283544, 0.303542, 0.335743, 0.31491, 0.357204, 0.434545, 0.410027, 0.404635],
    'Sents': [150, 147, 189, 84, 67, 84, 103, 95, 96],
    'Average sentence length': [15.8267, 16.1224, 12.8466, 26.3452, 34.8358, 27.7619, 21.3592, 23.5158, 23.375],
    '1-letter words': [416, 398, 361, 227, 262, 286, 213, 210, 265],
    '2-letter words': [430, 432, 424, 451, 559, 384, 334, 372, 371],
    '>15-letter words': [1, 2, 0, 1, 9, 3, 3, 6, 2],
    'Average word length': [3.68618, 3.66624, 3.63715, 4.26661, 4.39674, 4.58533, 4.75455, 4.61549, 4.29055],
    'Category': ['mystery', 'mystery', 'mystery', 'religion', 'religion', 'religion', 'editorial', 'editorial', 'editorial']
}

df = pd.DataFrame(data, index=['cl01', 'cl02', 'cl03', 'cd01', 'cd02', 'cd03', 'cb01', 'cb02', 'cb03'])

display(df)

## Étape 1 : choisir les variables prédictives

La première étape consiste à séparer les variables prédictives (celles qui nous aideront dans nos prédictions) de la variable dépendante (celle que l’on cherche à prédire). Nous identifions par ailleurs des variables redondantes : l’indice TTR étant un ratio entre les mots-formes uniques et le nombre de tokens, on peut retirer sans perte de notre modèle les variables *Tokens* et *Types*. Même traitement pour la variable *Sents* qui, à partir du moment où nous connaissons le nombre de tokens et la longueur moyenne d’une phrase, peut-être reconstituée.

Nos matrices deviennent :

In [ ]:
# predictive variables
X = df.drop(columns=['Tokens', 'Types', 'Sents', 'Category']).values

# target
y = df['Category'].values

# unique classes
classes = np.unique(y)

## Étape 2 : normaliser les données

On ne le répétera jamais assez, il est généralement recommandé de normaliser sa matrice de données : d’une part certaines variables sont sur des échelles très différentes, ce qui peut influencer l’analyse ; d’autre part les algorithmes sont plus robustes en travaillant avec des variables continues.

Pour *X*, on obtient :

In [ ]:
X_scaled = (X - X.mean(axis=0)) / X.std(axis=0)

## Étape 3 : calculer les moyennes

Afin de déterminer les variances inter-classes et intra-classes, il est nécessaire au préalable de calculer les moyennes globales et à l’intérieur des classes :

In [ ]:
mu = X_scaled.mean(axis=0)
means_per_class = {cls: X_scaled[y == cls].mean(axis=0) for cls in classes}

## Étape 4 : établir les matrices de dispersion

À cette étape, notre objectif est d’une part de révéler la manière dont les données varient à l’intérieur de leur classe et d’autre part comment les moyennes varient entre elles. Deux matrices seront calculées :

- la matrice de dispersion intra-classe ($S_W$), qui analyse la variance/covariance des données à l’intérieur de leur classe ;
- la matrice de dispersion inter-classe ($S_B$), qui mesure la variance entre les moyennes des classes.

Avant tout, initialisons des matrices carrées nulles :

In [ ]:
# initialization
S_W = np.zeros((X_scaled.shape[1], X_scaled.shape[1]))
S_B = np.zeros((X_scaled.shape[1], X_scaled.shape[1]))

### La matrice de dispersion intra-classe

Pour mesurer la variance à l’intérieur de chaque classe, nous devons nous intéresser à la dispersion des données par rapport à la moyenne dans chaque classe. La formule établit que :

$$
S_W = \sum_{k=1}^{K} \sum_{i \in C_k} (x_i - \mu_k)(x_i - \mu_k)^T
$$

Où :

- $K$ est le nombre de classes ;
- $C_k$ représente les données dans la classe $k$ ;
- $x_i$ est un vecteur de caractéristiques pour l’exemple $i$ dans la classe $k$ ;
- $\mu_k$ est la moyenne des caractéristiques dans la classe $k$.

### La matrice de dispersion inter-classe

Cette matrice se construit en analysant les moyennes des classes varient entre elles à partir de l’expression suivante :

$$
S_B = \sum_{k=1}^{K} N_k (\mu_k - \mu)(\mu_k - \mu)^T
$$

Où :

- $N_k$ est le nombre d'éléments dans la classe $k$ ;
- $\mu_k$ est la moyenne des caractéristiques de la classe $k$ ;
- $\mu$ est la moyenne globale de toutes les classes.

### Résolution des calculs

Les opérations se réalisent en une passe avec *Numpy* :

In [ ]:
for cls in classes:
    # matrix for a class
    X_cls = X_scaled[y == cls]
    mu_cls = means_per_class[cls]
    
    # within-class scatter
    S_W += np.dot((X_cls - mu_cls).T, (X_cls - mu_cls))
    
    # between-class scatter
    n_cls = X_cls.shape[0]
    S_B += n_cls * np.outer(mu_cls - mu, mu_cls - mu)

## Étape 5 : maximiser la séparation entre les classes

Si notre objectif était simplement de minimiser la variance intra-classe, $S_W$ seule suffirait à l’analyse. Cependant, dans la LDA, l’objectif est double : rechercher aussi les directions discriminantes en projetant les données sur un sous-espace où la séparation des classes est maximisée. Mathématiquement, cela revient à calculer :

$$
S_W^{-1}S_B
$$

En prenant l’inverse de $S_W$, nous réduisons l’impact de la dispersion intra-classe sur la séparation entre les classes. Plus précisément, cela permet de mettre en évidence les directions de l’espace vectoriel où les classes sont le mieux séparées, tout en tenant compte de la variance intra-classe par la diminution de l’influence des directions où les classes se chevauchent le plus.

Avec *Numpy*, on réalise l’opération simplement :

In [ ]:
S_W_inv = np.linalg.inv(S_W)
S_W_inv_S_B = np.dot(S_W_inv, S_B)

## Étape 6 : décomposer la matrice en éléments propres

Une fois la matrice $S_W^{-1}S_B$ obtenue, il reste à la décomposer en éléments propres afin de révéler :
- Les **valeurs propres** (*eigenvalues*), qui quantifient la force de chaque direction discriminante.
- Les **vecteurs propres** (*eigenvectors*), qui représentent les directions dans lesquelles la séparation des classes est maximisée.

Pour rappel, la décomposition en éléments revient à exprimer une matrice carrée comme le résultat de l’équation :

$$
A = PDP^{-1}
$$

Où :

- $P$ est la matrice des vecteurs propres ;
- $D$ la diagonale des valeurs propres ;
- $P^{-1}$ l’inverse de $P$ si $A$ est diagonalisable.

### Calculer les valeurs propres

Cela revient à extraire les racines du polynôme caractéristique de la matrice :

$$
P_M(x) = \det[M - x.I_n]
$$

### Calculer les vecteurs propres

Les vecteurs propres sont les vecteurs associés aux valeurs propres d’une matrice. Elles répondent au système d’équation ci-dessous où $I_n$ est la matrice identité et $\vec{X}$ un vecteur des racines du polynôme caractéristique :

$$
(M − \lambda I_n)\vec{X} = \vec{0}
$$

### Projeter les données sur un sous-espace vectoriel

La méthode `.linalg.eig()` de *Numpy* effectue les calculs en une ligne :

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(S_W_inv_S_B)

Il reste maintenant à sélectionner les vecteurs propres pour les valeurs propres les plus fortes. Pour cela, nous ordonnons d’abord les valeurs propres par ordre décroissant :

In [ ]:
# indices to sort the array?
idx = eigenvalues.argsort()[::-1]

# sorting & swiping out the imaginary part of the number
eigenvalues = np.real(eigenvalues[idx])
eigenvectors = np.real(eigenvectors[:, idx])

Ensuite, nous décidons de la dimensionnalité du sous-espace dans lequel nous souhaitons projeter les données, avec la contrainte qu’il peut, au maximum, avoir autant de dimensions que de classes $-1$ :

In [ ]:
# number of dimensions (max here: 2)
k = 2

# select the k eigenvectors
components = eigenvectors[:, :k]

Finalement, nous projetons les données sur le sous-espace :

In [ ]:
X_projected = np.dot(X_scaled, components)

# into a dataframe
df_projected = pd.DataFrame(X_projected, index=df.index, columns=['Component 1', 'Component 2'])
df_projected['Class'] = y

display(df_projected)

## Visualisation

Mettons que nous obtenons une nouvelle donnée pour laquelle nous souhaitons deviner la classe :

In [ ]:
# a new text
new_data = np.array([0.410961, 17.4314, 207, 498, 4, 3.72824]).reshape(1, -1)

# standardization
new_data_scaled = (new_data - new_data.mean()) / new_data.std()

# projection
new_data_projected = np.dot(new_data_scaled, components)

# into a dataframe
df_new = pd.DataFrame(new_data_projected, columns=['Component 1', 'Component 2'])
df_new['Class'] = 'Unknown'

# concatenation
df_projected = pd.concat([df_projected, df_new], ignore_index=True)

Preuve que la LDA a bien fonctionné, nous visualisons dans un nuage de points des partitions distinctes qui nous incitent à attribuer la classe *mystery* au nouveau texte :

In [ ]:
plt.figure(figsize=(8, 6))

sns.scatterplot(
    data=df_projected,
    x='Component 1',
    y='Component 2',
    hue='Class',
    palette='Set2',
    style='Class',
    s=100,
    alpha=0.7
)

plt.title('Linear Discriminant Analysis (LDA)', fontsize=16)
plt.xlabel('Component 1', fontsize=12)
plt.ylabel('Component 2', fontsize=12)
plt.legend(title='Class', title_fontsize='13', fontsize='11', loc='best')

plt.xlim(-2, 2)
plt.ylim(-1, 1)
plt.grid(True, linestyle='--', alpha=0.5)

plt.show()

## Établir un modèle discriminant

Si elle s’interprète facilement, la preuve graphique n’est pas suffisante pour parler de modèle de classification. Une fois les données projetées sur un sous-espace de $k-1$ classes, nous pouvons construire un modèle basé sur les fonctions discriminantes de la LDA :

$$
g_k(x) = W_k \cdot x + b_k
$$

Où :

- $W_k$ figure les coefficients (ou poids) ;
- $b_k$ le biais.

### Calculer les coefficients

La matrice des coefficients se calcule comme :

$$
W_k = S_W^{−1} \cdot (\mu_k − \mu)
$$

Où :

- $\mu_k$ la moyenne de la classe $k$ ;
- $\mu$ la moyenne globale.

Nous avons besoin de connaître en premier lieu les moyennes globale et de chaque classe dans l’espace projeté :

In [ ]:
# overall mean in the projected space
mean_overall = np.mean(X_projected, axis=0)

# mean of each class in the projected space
means_per_class = [np.mean(X_projected[y == c], axis=0) for c in classes]

Ensuite, nous recalculons l’inverse de la matrice de dispersion intra-classe dans l’espace projeté. Dans ce contexte, $S_W$ se comprend comme la matrice de covariance des dimensions réduites obtenues après projection sur les vecteurs propres. Comme `X_projected` est une matrice contenant en lignes les observations et en colonnes les dimensions, il nous suffit de la transposer pour obtenir la covariance des dimensions :

In [ ]:
S_W_projected = np.cov(X_projected.T)
S_W_inv = np.linalg.inv(S_W_projected)

### Calculer le biais

Le biais est calculé par :

$$
b_k = -\frac{1}{2} \mu_k^T \cdot S_W^{−1} \cdot \mu_k + \log⁡(\pi_k)
$$

Où $\pi_k$ est le *prior* (probabilité a priori) de la classe $k$. L’existence de ce *prior* se justifie par la possibilité de déséquilibre de représentation d’une classe dans les données d’apprentissage. Pour éviter ce risque, le modèle prend en compte les connaissances a priori. De manière naïve, en présence de trois classes nous estimons le *prior* à 0.33 pour chacune.

Dans la pratique, il faut considérer ce mécanisme comme une possibilité de représenter les connaissances a priori : si nous sommes certain·es qu’une classe se rencontre davantage dans la nature, il nous faudra bouger le curseur en conséquence. Et si en revanche les données sont parfaitement équilibrées, comme dans notre exemple où nous avons trois représentants de chaque classe, les *priors* n’auront aucun impact et pourraient être ignorées. Nous tenons toutefois à les conserver :

In [ ]:
priors = [np.sum(y == c) / len(y) for c in classes]

### Construire le modèle

Tous les éléments étant en place, nous pouvons construire le modèle en reprenant la formule énoncée plus haut :

In [ ]:
coefficients = []
for k, class_mean in enumerate(means_per_class):
    W_k = np.dot(S_W_inv, class_mean - mean_overall)
    b_k = -0.5 * np.dot(class_mean, np.dot(S_W_inv, class_mean)) + np.log(priors[k])
    coefficients.append((W_k, b_k))

### Effectuer des prédictions

Il est temps d’utiliser le modèle pour prédire la classe de notre nouvelle observation, selon l'expression :

$$
\hat{y} = \arg \max_k g_k(x)
$$

La formule se comprend facilement : nous appliquons la fonction discriminante du modèle ajusté à chaque classe et choisissons la classe correspondant à l’argument qui maximise la fonction discriminante.

In [ ]:
scores = [
    np.dot(new_data_projected, W_k) + b_k
    for W_k, b_k in coefficients
]

print(
    f"Score : {np.max(scores):.4f}",
    f"Catégorie : {classes[np.argmax(scores)]}",
    sep="\n"
)

Cerise sur le gâteau, il est possible de ressortir la probabilité associée à chaque classe grâce à la fonction de densité a posteriori :

In [ ]:
# according to Bayes theorem
exps = np.exp(scores - np.max(scores))
probabilities = exps / np.sum(exps)

for class_name, p in zip(classes, probabilities):
    print(f'{class_name} : {p[0]:.2%}')

## Évaluer la performance du modèle